In [1]:
import numpy as np
from typing import Tuple, Optional

In [42]:
def identify_leading_edge(waveform: np.ndarray, tau: float) -> Tuple[Optional[int], Optional[int]]:
    """
    Identify the startgate and stopgate of a waveform.
    From algo described here: https://climate.esa.int/sites/default/files/Sea_State_cci_ATBD_v1.1-signed_0.pdf

    :param np.ndarray waveform: numpy array of waveform values
    :param float tau: time spacing between consecutive gates in seconds
    :return: Tuple containing:
      - startgate: index of the start gate (Optional[int])
      - stopgate: index of the stop gate (Optional[int])
    :rtype: Tuple[Optional[int], Optional[int]]

    """
    # The waveform is normalised with normalisation factor N, 
    # where N = 1.3 * median(waveform)
    N: float = 1.3 * np.median(waveform)
    normalized_waveform: np.ndarray = waveform / N

    # The leading edge starts when the normalised waveform has a 
    # rise of 0.01 units compared to the previous gate (startgate)
    startgate: int | None = None
    for i in range(1, len(normalized_waveform)):
        if normalized_waveform[i] - normalized_waveform[i-1] >= 0.01:
            startgate = i
            break

    if startgate is None:
        return None, None  # No valid startgate found

    # At this point, the leading edge is considered valid if, for at least four gates 
    # after startgate, it does not decrease below 0.1 units (10% of the normalised power).
    valid_leading_edge: bool = (
        startgate + 4 < len(normalized_waveform) and
        np.all(normalized_waveform[startgate:startgate + 5] >= 0.1)
    )

    if not valid_leading_edge:
        return None, None  # Leading edge is not valid

    # The end of the leading edge (stopgate) is fixed at the first gate 
    # in which the derivative changes sign (i.e. the signal start decreasing 
    # and the trailing edge begins), if the change of sign is kept 
    # for the following 3 gates.
    stopgate: int | None = None
    for i in range(startgate + 1, len(normalized_waveform) - 3):
        # Calculate the derivative
        derivative: float = normalized_waveform[i + 1] - normalized_waveform[i]
        if derivative < 0:  # Start of decrease
            # Check if the derivative stays negative for the next 3 gates
            if (normalized_waveform[i + 2] - normalized_waveform[i + 1] < 0 and
                normalized_waveform[i + 3] - normalized_waveform[i + 2] < 0):
                stopgate = i
                break

    return startgate, stopgate

In [43]:
if __name__ == "__main__":
    # Simulated waveform data
    waveform: np.ndarray = np.array([0.0, 0.0, 0.0, 0.0001, 0.0015, 0.002, 0.004, 0.01, 0.02, 0.05, 0.15, 0.1, 0.08, 0.07, 0.03, 0.01, 0.0])
    tau: float = 3.125e-9  # Time spacing in seconds

    (startgate, stopgate) = identify_leading_edge(waveform, tau)

    print(f"Startgate: {startgate}, Stopgate: {stopgate}")

Startgate: 4, Stopgate: 10
